In [78]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

# ================= 绘图风格设置 =================
fonts = ['SimHei', 'Microsoft YaHei', 'SimSun', 'Arial Unicode MS']
found_font = False
for font in fonts:
    try:
        mpl.rcParams['font.sans-serif'] = [font] + mpl.rcParams['font.sans-serif']
        found_font = True
        break
    except:
        continue

mpl.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')


In [79]:

# ================= 智能路径设置 =================
# 目标：找到 summary_stats.csv 所在的目录，并将 figures 保存到那里

current_cwd = os.getcwd()
print(f"当前 Jupyter 工作目录: {current_cwd}")

# 候选路径列表 (相对于当前工作目录)
candidates = [
    ".",  # 当前目录
    "results_analysis/deepsort_metrics",  # 如果在项目根目录运行
    "deepsort_metrics", # 如果在 results_analysis 运行
    "../results_analysis/deepsort_metrics" # 其他情况
]

target_dir = None
csv_file = "summary_stats.csv"

for path in candidates:
    full_path = os.path.join(current_cwd, path)
    if os.path.exists(os.path.join(full_path, csv_file)):
        target_dir = os.path.abspath(full_path)
        break

if target_dir is None:
    # 如果都找不到，尝试使用绝对路径 (作为最后的手段)
    # 假设用户路径结构固定
    absolute_fallback = r"D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics"
    if os.path.exists(os.path.join(absolute_fallback, csv_file)):
        target_dir = absolute_fallback
    else:
        raise FileNotFoundError(f"无法找到 {csv_file}，请检查工作目录或文件位置。")

print(f"定位到数据目录: {target_dir}")

BASE_DIR = target_dir
CSV_PATH = os.path.join(BASE_DIR, "summary_stats.csv")
FIG_DIR = os.path.join(BASE_DIR, "figures")

# 创建 figures 目录
if not os.path.exists(FIG_DIR):
    os.makedirs(FIG_DIR)
    print(f"创建目录: {FIG_DIR}")
else:
    print(f"目录已存在: {FIG_DIR}")


当前 Jupyter 工作目录: D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics
定位到数据目录: D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics
目录已存在: D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics\figures


In [86]:

# ================= 读取数据 =================
df = pd.read_csv(CSV_PATH)
print(df.describe())

        sequence  num_tracks  avg_length  max_length  num_fragments  \
count  21.000000   21.000000   21.000000   21.000000      21.000000   
mean   10.000000   11.952381  180.117201  309.190476      23.273885   
std     6.204837    4.914023  114.547339  194.520595      19.468821   
min     0.000000    4.000000   50.714286   76.000000       4.000000   
25%     5.000000    8.000000   96.250000  187.000000      11.166667   
50%    10.000000   12.000000  164.785714  259.000000      19.285714   
75%    15.000000   14.000000  218.882353  357.000000      28.285714   
max    20.000000   24.000000  491.190476  803.000000      89.238095   

        avg_speed  avg_acceleration  
count   21.000000         21.000000  
mean    86.788131          0.663358  
std     34.747614          0.682315  
min     12.178000         -0.886280  
25%     64.723485          0.153074  
50%     86.162526          0.574373  
75%    101.533838          1.162063  
max    156.329190          1.966874  


In [81]:
print("已加载 summary_stats.csv")
print(df.head())


已加载 summary_stats.csv
   sequence  num_tracks  avg_length  max_length  num_fragments   avg_speed  \
0         0          16   81.500000         148       6.062500  101.533838   
1         1          17  218.882353         355      29.941176  121.951294   
2         2          12   92.166667         187      11.166667   86.162526   
3         3           4   96.250000         115      13.500000   63.518764   
4         4           7  158.000000         230      27.857143  124.697391   

   avg_acceleration  
0          1.434673  
1          0.023859  
2         -0.886280  
3          0.574373  
4          1.391665  


In [82]:

# ================= 可视化函数定义 =================
def save_bar(x, y, title, xlabel, ylabel, filename, color='steelblue'):
    plt.figure(figsize=(10, 6))
    plt.bar(x, y, color=color, alpha=0.8, edgecolor='black')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    save_path = os.path.join(FIG_DIR, filename)
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"已保存: {save_path}")

def save_hist(data, title, xlabel, ylabel, filename, color='mediumseagreen'):
    plt.figure(figsize=(8, 6))
    plt.hist(data, bins=15, color=color, alpha=0.7, edgecolor='black')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    save_path = os.path.join(FIG_DIR, filename)
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"已保存: {save_path}")

def save_scatter(x, y, title, xlabel, ylabel, filename, color='crimson'):
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, color=color, alpha=0.8, s=80, edgecolor='white')
    
    if len(x) > 1:
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        plt.plot(x, p(x), "b--", alpha=0.6, label=f'趋势线 (y={z[0]:.2f}x+{z[1]:.2f})')
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    save_path = os.path.join(FIG_DIR, filename)
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"已保存: {save_path}")


In [83]:

# ================= 1️⃣ 基础准确性分析 =================

# 1.1 各序列轨迹数量分布
save_bar(
    df["sequence"],
    df["num_tracks"],
    "各测试序列轨迹数量分布",
    "序列 ID",
    "轨迹数量 (个)",
    "1_num_tracks_per_sequence.png",
    color='#4c72b0'
)

# 1.2 平均轨迹长度分布 (反映跟踪持续性)
save_hist(
    df["avg_length"],
    "平均轨迹长度分布直方图",
    "平均轨迹长度 (帧数)",
    "频数 (序列数量)",
    "2_avg_length_hist.png",
    color='#55a868'
)

# 1.3 运动稳定性分析 (平均速度分布)
save_hist(
    df["avg_speed"],
    "目标平均运动速度分布",
    "平均速度 (像素/帧)",
    "频数",
    "3_avg_speed_hist.png",
    color='#c44e52'
)


已保存: D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics\figures\1_num_tracks_per_sequence.png
已保存: D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics\figures\2_avg_length_hist.png
已保存: D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics\figures\3_avg_speed_hist.png


In [84]:

# ================= 2️⃣ 遮挡影响量化研究 =================

# 2.1 场景密度与轨迹碎片化的关系
save_scatter(
    df["num_tracks"],
    df["num_fragments"],
    "场景密度与轨迹碎片化关系分析 (遮挡影响)",
    "场景密度 (轨迹数量/序列)",
    "平均碎片数量 (次/轨迹)",
    "4_density_vs_fragmentation.png",
    color='#8172b3'
)

# 2.2 场景密度与跟踪完整性的关系
save_scatter(
    df["num_tracks"],
    df["avg_length"],
    "场景密度与平均跟踪长度关系分析",
    "场景密度 (轨迹数量/序列)",
    "平均轨迹长度 (帧数)",
    "5_density_vs_length.png",
    color='#ccb974'
)


已保存: D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics\figures\4_density_vs_fragmentation.png
已保存: D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics\figures\5_density_vs_length.png


In [85]:

# ================= 3️⃣ 综合数据导出 =================
correlation_matrix = df[['num_tracks', 'avg_length', 'num_fragments', 'avg_speed']].corr()
print("\n相关系数矩阵 (Correlation Matrix):")
print(correlation_matrix)

correlation_matrix.to_csv(os.path.join(BASE_DIR, "correlation_analysis.csv"))

print("\n分析完成！")
print(f"可视化图表已保存至: {FIG_DIR}")


相关系数矩阵 (Correlation Matrix):
               num_tracks  avg_length  num_fragments  avg_speed
num_tracks       1.000000    0.378013       0.365451   0.523794
avg_length       0.378013    1.000000       0.945887   0.300457
num_fragments    0.365451    0.945887       1.000000   0.434146
avg_speed        0.523794    0.300457       0.434146   1.000000

分析完成！
可视化图表已保存至: D:\Data\25-26一\图像工程\work\results_analysis\deepsort_metrics\figures
